In this kernel I want to illustrate how I do come up with meaningful preprocessing when building deep learning NLP models. 

I start with two golden rules:

1.  **Don't use standard preprocessing steps like stemming or stopword removal when you have pre-trained embeddings** 

Some of you might used standard preprocessing steps when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc. 
The reason is simple: You loose valuable information, which would help your NN to figure things out.  

2. **Get your vocabulary as close to the embeddings as possible**

I will focus in this notebook, how to achieve that. For an example I take the GoogleNews pretrained embeddings, there is no deeper reason for this choice.

We start with a neat little trick that enables us to see a progressbar when applying functions to a pandas Dataframe

In [1]:
import json
from tqdm import tqdm
tqdm.pandas()


Lets load our data

In [2]:
import json
with open('inverted.json') as f:
    for line in f:
        pass
invert = json.loads(line)

I will use the following function to track our training vocabulary, which goes through all our text and counts the occurance of the contained words. 

In [3]:
vocab = {}
with open('./analysis/vocab') as f:
    for line in f:
        line = line.split()
        vocab[line[0]] = int(line[1])

So lets populate the vocabulary and display the first 5 elements and their count. Note that now we can use progess_apply to see progress bar

Next we import the embeddings we want to use in our model later. For illustration I use GoogleNews here.

In [4]:
from gensim.models import KeyedVectors

news_path = './embedding/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

/home/kyhuang/anaconda3/envs/NLP_p1/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


Next I define a function that checks the intersection between our vocabulary and the embeddings. It will output a list of out of vocabulary (oov) words that we can use to improve our preprocessing

In [5]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [6]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 1048921/1048921 [00:01<00:00, 603618.53it/s]


Found embeddings for 11.24% of vocab
Found embeddings for  70.69% of all text


Ouch only 24% of our vocabulary will have embeddings, making 21% of our data more or less useless. So lets have a look and start improving. For this we can easily have a look at the top oov words.

In [7]:
oov[:20]

[('.', 28644184),
 (',', 25669672),
 ('to', 11765243),
 ('a', 10440410),
 ('and', 10298461),
 ('of', 9688997),
 ('-', 6673699),
 ('”', 3175358),
 (':', 2608659),
 ('—', 1516752),
 (')', 1400229),
 ('(', 1373208),
 ("'", 1357140),
 ('"', 880445),
 ('?', 807212),
 ('000', 618443),
 ('it’s', 516104),
 (';', 477945),
 ('10', 369289),
 ('“i', 325595)]

On first place there is "to". Why? Simply because "to" was removed when the GoogleNews Embeddings were trained. We will fix this later, for now we take care about the splitting of punctuation as this also seems to be a Problem. But what do we do with the punctuation then - Do we want to delete or consider as a token? I would say: It depends. If the token has an embedding, keep it, if it doesn't we don't need it anymore. So lets check:

In [8]:
'?' in embeddings_index

False

In [9]:
'&' in embeddings_index

True

Interesting. While "&" is in the Google News Embeddings, "?" is not. So we basically define a function that splits off "&" and removes other punctuation.

In [10]:
vocab_temp = vocab.copy()
invert_temp = invert.copy()

In [11]:
def clean_text(vocab,inverted):
    kkey = list(vocab.keys())
    qq = []
    for key in kkey:
        x = str(key)
        for punct in "/-'":
            x = x.replace(punct, '')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’‘':
            x = x.replace(punct, '')
        
        if(x!=key):
            if(len(x)>0):
                if(x not in vocab and x not in inverted):
                    inverted[x] = {'docs':{}}
                    vocab[x] = 0
                elif(x not in vocab):
                    vocab[x] = 0
                    print('--',x)
                elif(x not in inverted):
                    inverted[x] = {'docs':{}}
                    print('==',x)
                    
                vocab[x] += vocab[key]
                qq.append((key,x))
                for w,n in inverted[key]['docs'].items():
                    try:
                        inverted[x]['docs'][w] += n
                    except:
                        inverted[x]['docs'][w] = n
            del vocab[key]
            del inverted[key]
    return qq

In [12]:
change = clean_text(vocab,invert)

In [13]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 843238/843238 [00:01<00:00, 617928.06it/s]


Found embeddings for 14.16% of vocab
Found embeddings for  84.93% of all text


Nice! We were able to increase our embeddings ratio from 24% to 57% by just handling punctiation. Ok lets check on thos oov words.

In [14]:
oov[:10]

[('to', 11788929),
 ('a', 10516905),
 ('and', 10341307),
 ('of', 9694380),
 ('—', 1517033),
 ('000', 618627),
 ('10', 370043),
 ('30', 308695),
 ('11', 232210),
 ('12', 190189)]

Hmm seems like numbers also are a problem. Lets check the top 10 embeddings to get a clue.

In [15]:
for i in range(10):
    print(embeddings_index.index2entity[i])

</s>
in
for
that
is
on
##
The
with
said


hmm why is "##" in there? Simply because as a reprocessing all numbers bigger tha 9 have been replaced by hashs. I.e. 15 becomes ## while 123 becomes ### or 15.80€ becomes ##.##€. So lets mimic this preprocessing step to further improve our embeddings coverage

In [18]:
import re

def clean_numbers(vocab,inverted):
    kkey = list(vocab.keys())
    for key in kkey:
        
        x = str(key)
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
        
        if(x!=key):
            if(len(x)>0):
                if(x not in vocab or x not in inverted):
                    inverted[x] = {'docs':{}}
                    vocab[x] = 0
                vocab[x] += vocab[key]
                print(key,x)
                for w,n in inverted[key]['docs'].items():
                    try:
                        inverted[x]['docs'][w] += n
                    except:
                        inverted[x]['docs'][w] = n
            del vocab[key]
            del inverted[key]

In [20]:
clean_numbers(vocab,invert)

000 ###
10 ##
30 ##
11 ##
12 ##
20 ##
15 ##
703 ###
900 ###
25 ##
13 ##
18 ##
2012 ####
14 ##
50 ##
16 ##
100 ###
2014 ####
17 ##
500 ###
2016 ####
24 ##
2013 ####
40 ##
21 ##
19 ##
2015 ####
22 ##
28 ##
2011 ####
23 ##
27 ##
202 ###
26 ##
2010 ####
301 ###
29 ##
200 ###
45 ##
2008 ####
35 ##
31 ##
60 ##
2009 ####
300 ###
400 ###
90 ##
800 ###
55 ##
32 ##
2007 ####
70 ##
34 ##
33 ##
80 ##
36 ##
600 ###
990 ###
48 ##
38 ##
540 ###
2005 ####
37 ##
2000 ####
42 ##
2006 ####
75 ##
39 ##
700 ###
41 ##
44 ##
43 ##
47 ##
2004 ####
49 ##
52 ##
46 ##
51 ##
250 ###
95 ##
2001 ####
2017 ####
53 ##
54 ##
65 ##
56 ##
99 ##
150 ###
58 ##
59 ##
57 ##
2003 ####
2002 ####
14th ##th
1990s ####s
240 ###
571 ###
1980s ####s
999 ###
410 ###
66 ##
1000 ####
1999 ####
85 ##
1970s ####s
00 ##
64 ##
05 ##
62 ##
1996 ####
20th ##th
1998 ####
2018 ####
10th ##th
63 ##
15th ##th
102 ###
72 ##
68 ##
1960s ####s
19th ##th
350 ###
1200 ####
1100 ####
1992 ####
21st ##st
777 ###
1300 ####
61 ##
120 ###
67 ##
11th ##t

840 ###
811 ###
13065 #####
564 ###
1701 ####
5301 ####
75th ##th
588 ###
12100 #####
817 ###
462 ###
878 ###
1935 ####
421 ###
14800 #####
451 ###
622 ###
1916 ####
42nd ##nd
787 ###
4304 ####
819 ###
9601 ####
9535 ####
543 ###
487 ###
859 ###
353 ###
8601 ####
10600 #####
1934 ####
7759 ####
1928 ####
1913 ####
8857 ####
446 ###
346 ###
448 ###
51st ##st
1632 ####
1925 ####
1923 ####
70th ##th
6372 ####
1921 ####
11900 #####
hart90 hart##
523 ###
577 ###
1015 ####
13800 #####
383 ###
10px ##px
382 ###
532 ###
12900 #####
60th ##th
884 ###
538 ###
1927 ####
542 ###
704 ###
366 ###
803 ###
14300 #####
1630 ####
8477 ####
291 ###
4070 ####
791 ###
714 ###
1922 ####
920 ###
837 ###
631 ###
623 ###
869 ###
552 ###
607 ###
11800 #####
706 ###
12500 #####
11000 #####
39th ##th
10700 #####
1780 ####
477 ###
868 ###
48th ##th
38th ##th
536 ###
44th ##th
10877 #####
452 ###
36441 #####
879 ###
10800 #####
41st ##st
2026 ####
381 ###
1609 ####
716 ###
919 ###
797 ###
386 ###
612 ###
898 ###
15

1505 ####
6630 ####
6972 ####
873 ###
20151 #####
4808 ####
1602 ####
0001 ####
3550 ####
005 ###
2926 ####
1206 ####
846 ###
20715 #####
1618 ####
2425 ####
10401 #####
22311 #####
76th ##th
4941 ####
1411 ####
1750 ####
6405 ####
1216 ####
7975 ####
2270 ####
4x400 4x###
43008 #####
4343 ####
1217 ####
1103 ####
1214 ####
973 ###
2903 ####
22303 #####
9701 ####
1313 ####
13053 #####
20017 #####
1014 ####
9518 ####
21029 #####
3637 ####
852 ###
8881 ####
86th ##th
20194 #####
2567 ####
9444 ####
14515 #####
4025 ####
22556 #####
1859 ####
19100 #####
9410 ####
4356 ####
5890 ####
2796 ####
0394 ####
6612 ####
1324 ####
83rd ##rd
3999 ####
3174 ####
1620 ####
1860s ####s
1848 ####
12001 #####
4125 ####
7671 ####
1304 ####
20877 #####
1714 ####
6019 ####
1857 ####
2480 ####
3713 ####
20705 #####
89th ##th
1408 ####
2999 ####
20015 #####
22202 #####
45745 #####
6491 ####
6252 ####
2650 ####
22150 #####
030 ###
8599 ####
015 ###
8186 ####
1621 ####
992 ###
996 ###
4729 ####
22026 #####
56

5018 ####
7208 ####
7802 ####
6003 ####
3407 ####
9573 ####
45131 #####
20842 #####
0113 ####
5723 ####
1665 ####
1341 ####
8151 ####
7020 ####
8820 ####
8407 ####
6420 ####
9144 ####
5408 ####
6260 ####
9713 ####
5011 ####
1260 ####
4252 ####
3708 ####
7110 ####
3806 ####
4202 ####
5013 ####
20736 #####
4377 ####
22125 #####
4129 ####
3425 ####
7917 ####
24950 #####
3214 ####
5363 ####
8906 ####
3213 ####
2582 ####
3717 ####
2810 ####
2119 ####
3204 ####
4102 ####
0984 ####
7206 ####
14604 #####
3062 ####
3504 ####
2038 ####
1027 ####
12333 #####
5610 ####
027 ###
4903 ####
4203 ####
1774 ####
3883 ####
3934 ####
9302 ####
076 ###
2616 ####
19355 #####
4115 ####
104th ###th
2516 ####
1326 ####
1736 ####
9811 ####
2832 ####
2135 ####
5113 ####
1753 ####
30pm ##pm
6503 ####
8324 ####
3814 ####
8110 ####
6303 ####
6432 ####
8097 ####
9774 ####
7814 ####
20700 #####
3625 ####
9338 ####
4950 ####
1060 ####
1500s ####s
2806 ####
4220 ####
6703 ####
9705 ####
8203 ####
0750 ####
6513 ####
83

6278 ####
13405 #####
9623 ####
13401 #####
11720 #####
2656 ####
11512 #####
11914 #####
10867 #####
1253 ####
2540 ####
11216 #####
42395 #####
5218 ####
300px ###px
13869 #####
7188 ####
9948 ####
7798 ####
3823 ####
1080p ####p
7725 ####
5222 ####
4031 ####
10821 #####
6840 ####
13710 #####
5028 ####
5413 ####
7346 ####
12602 #####
8322 ####
3596 ####
1588 ####
2730 ####
10228 #####
5939 ####
6725 ####
8114 ####
9002 ####
8392 ####
6721 ####
4187 ####
0777 ####
4330 ####
4929 ####
11601 #####
4617 ####
7878 ####
10802 #####
12507 #####
14704 #####
10704 #####
0109 ####
4815 ####
13110 #####
2128 ####
2527 ####
15108 #####
13609 #####
14511 #####
9233 ####
12510 #####
9817 ####
13305 #####
10906 #####
10306 #####
15251 #####
8224 ####
9113 ####
2186 ####
11304 #####
77px ##px
11606 #####
whole30 whole##
5335 ####
7398 ####
5630 ####
1581 ####
8024 ####
14811 #####
1460 ####
13106 #####
7543 ####
9551 ####
1773 ####
2526 ####
22600 #####
8014 ####
5521 ####
3346 ####
7408 ####
12214 

...

944–59 ###–##
218–25 ###–##
710–19 ###–##
42–50 ##–##
739–46 ###–##
358–61 ###–##
576–600 ###–###
7024c ####c
174m ###m
8448736 #####
8448746 #####
8448755 #####
8448769 #####
100dayloans ###dayloans
uno2013 uno####
390bf1dd ###bf1dd
1fcbecebdd74 1fcbecebdd##
kids—21 kids—##
36pm ##pm
h33t h##t
3427148 #####
25–year ##–year
358c758b ###c###b
4c37 4c##
f4c7132e15ee f4c####e##ee
16—came ##—came
2w46 2w##
068fb7b4a845 ###fb7b4a###
8d3fe977 8d3fe###
761e ###e
16ba1eef2390 ##ba1eef####
8458693 #####
v22 v##
p248 p###
341–42 ###–##
ab2501 ab####
be24 be##
162b4c8a ###b4c8a
1968– ####–
8463837 #####
8463923 #####
8463858 #####
8463868 #####
8463882 #####
8463892 #####
8463907 #####
740th ###th
949–50 ###–##
827th ###th
acres–100 acres–###
4336055 #####
1420s ####s
11–which ##–which
13–but ##–but
50w ##w
83b ##b
1200nw ####nw
13357d #####d
12430b #####b
42316 #####
8466171 #####
8466172 #####
8466173 #####
8466175 #####
£292 £###
x900b x###b
hw40es hw##es
f5d955cc f5d###cc
905b ###b
e6ea6a60f6

ebj2012 ebj####
619174739781775361 #####
619169871260164096 #####
619166443066646528 #####
618472716375564288 #####
500° ###°
1000°f ####°f
16l ##l
17h ##h
17l ##l
18j ##j
28g ##g
6067a ####a
314b ###b
42193 #####
41854 #####
40888 #####
44852 #####
20229 #####
111noonherst ###noonherst
501milewalk ###milewalk
2012–2013 ####–####
my2131 my####
268e ###e
b621 b###
b55e495e9b78 b##e###e9b##
619718532268445696 #####
619879799536918528 #####
619880734933684224 #####
21grandslams ##grandslams
wimbledon2015 wimbledon####
2014a ####a
tunica06dv1 tunica##dv1
tunica06mv1 tunica##mv1
267000 #####
2008…sba ####…sba
thisis50 thisis##
1989tourwashingtondc ####tourwashingtondc
15an ##an
£48 £##
90002188 #####
405′ ###′
366′ ###′
452′ ###′
404′ ###′
431′ ###′
441′ ###′
420′ ###′
371′ ###′
372′ ###′
400′ ###′
351′ ###′
376′ ###′
385′ ###′
421′ ###′
399′ ###′
445′ ###′
433′ ###′
390′ ###′
377′ ###′
411′ ###′
402′ ###′
563792055651270656 #####
999999999999 #####
500746781622562816 #####
aylorswift13 ayl

707620187734982656 #####
708081559862120452 #####
60s—perhaps ##s—perhaps
3407a ####a
46755 #####
724s ###s
515m ###m
97m ##m
475m ###m
075m ###m
580m ###m
315m ###m
320m ###m
050m ###m
380m ###m
031m ###m
603mm ###mm
708190667281793024 #####
80d ##d
ku2016champs ku####champs
708334815372828672 #####
674021406426980352 #####
1990s…those ####s…those
4a6332ce 4a####ce
e443 e###
47cb ##cb
b0ca00411fe1 b0ca#####fe1
hb1524 hb####
sb1150 sb####
709564825500848128 #####
709598270402596864 #####
1841—the ####—the
r100 r###
jimward21 jimward##
709918638816550912 #####
622242105033764864 #####
958848 #####
267th ###th
185cd514 ###cd###
d71f d##f
a65b a##b
587e721fb231 ###e###fb###
1930s—i ####s—i
21—examining ##—examining
fb6547 fb####
710453854014279680 #####
r136a1 r###a1
710660117138640896 #####
112–58 ###–##
404–238 ###–###
39921 #####
s712 s###
1801d ####d
14016c #####c
12763 #####
7592a ####a
8240b ####b
100096 #####
955th ###th
a10wbb a##wbb
a10champs a##champs
736–37 ###–##
889–91 ###–##

1830b ####b
rayophotography13 rayophotography##
823221915171061760 #####
obama17 obama##
242196 #####
259636 #####
834017335669358593 #####
835194156175798277 #####
1999–to ####–to
835027942262484992 #####
835048458645970946 #####
835558339002134528 #####
835343752386514944 #####
jaredjeffrey614 jaredjeffrey###
pmullins14 pmullins##
300126215931 #####
836100391314075649 #####
664453 #####
836124739735326720 #####
kingj9943 kingj####
836078946039042049 #####
836357784031510529 #####
836402392035901440 #####
836414672731525120 #####
836382208466235393 #####
sb55 sb##
newspaper15 newspaper##
835845404734414848 #####
61016 #####
836645013685592065 #####
iriseup99 iriseup##
835004560947150848 #####
834964849264205825 #####
836741000022347776 #####
837099085660360704 #####
1925–30 ####–##
shatty22 shatty##
837298505672687616 #####
837323083589566467 #####
fatmeh32 fatmeh##
andrewsholt10 andrewsholt##
g304 g###
206w ###w
41429 #####
12434b #####b
42085lot #####lot
34877 #####
46887 #####
4599

72oz ##oz
1503s ####s
2009…its ####…its
1839s ####s
‘4000 ‘####
w20167s w#####s
officialsf49ers officialsf##ers
f001 f###
ar2339s ar####s
20ss ##ss
295s ###s
snl40 snl##
90125 #####
69′ ##′
18×24 ##×##
50‑50′ ##‑##′
kfp48 kfp##
wizards2010 wizards####
26y ##y
47daysofhitman ##daysofhitman
3191s ####s
i360s i###s
fifa15 fifa##
7042s ####s
16…he ##…he
2001—about ####—about
1651s ####s
300gb ###gb
15… ##…
‘1031 ‘####
early90s early##s
fn2187 fn####
1240s ####s
faith47s faith##s
je11s je##s
‎24hours‬ ‎##hours‬
192s ###s
v90 v##
1973–certainly ####–certainly
211s ###s
i09s i##s
kukiz15 kukiz##
alevine843 alevine###
‘666 ‘###
‘meet24 ‘meet##
15–to–watch ##–to–watch
farooksyed49 farooksyed##
iluvusa1132 iluvusa####
101sts ###sts
99¢ ##¢
c86 c##
1917–1963 ####–####
87… ##…
article19s article##s
c57 c##
2014…andersons ####…andersons
87s ##s
360is ###is
72… ##…
c84 c##
c77 c##
superfly1069 superfly####
£un35 £un##
80s—well ##s—well
‘bh34 ‘bh##
weather5280s weather####s
b330s b###s
96ers ##ers
gl

In [21]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 812155/812155 [00:01<00:00, 620500.06it/s]

Found embeddings for 14.76% of vocab
Found embeddings for  87.25% of all text


Nice! Another 3% increase. Now as much as with handling the puntuation, but every bit helps. Lets check the oov words

Ok now we  take care of common misspellings when using american/ british vocab and replacing a few "modern" words with "social media" for this task I use a multi regex script I found some time ago on stack overflow. Additionally we will simply remove the words "a","to","and" and "of" since those have obviously been downsampled when training the GoogleNews Embeddings. 


In [16]:
def replace_typical_misspell(vocab,inverted,key,re_word):
    if(key in vocab or key in inverted):
        print('o',key,re_word)
        for x in re_word:
            if(x not in vocab and x not in inverted):
                inverted[x] = {'docs':{}}
                vocab[x] = 0
            elif(x not in vocab):
                vocab[x] = 0
                print('--',x)
            elif(x not in inverted):
                inverted[x] = {'docs':{}}
                print('==',x)
            vocab[x] += vocab[key]
            for w,n in inverted[key]['docs'].items():
                try:
                    inverted[x]['docs'][w] += n
                except:
                    inverted[x]['docs'][w] = n

        del vocab[key]
        del inverted[key]

In [17]:
mispell_dict = {'didnt':['did','not'],
                'doesnt':['does','not'],
                'isnt':['is','not'],
                'shouldnt':['should','not'],
                'wasnt':['was','not'],
                'hasnt':['has','not'],
                '‘i':['i'],
                'theatre':['theater'],
                'cancelled':['canceled'],
                'organisation':['organization'],
                'labour':['labor'],
                'favourite':['favorite'],
                'travelling':['traveling'],
                'washingtons':['washington'],
                'marylands':['maryland'],
                'chinas':['china'],
                'russias':['russia'],
                '‘the':['the'],
                'irans':['iran'],
                'dulles':['dulle']
                }


In [18]:
for key,re_word in mispell_dict.items():
    replace_typical_misspell(vocab,invert,key,re_word)

o didnt ['did', 'not']
o doesnt ['does', 'not']
o isnt ['is', 'not']
o shouldnt ['should', 'not']
o wasnt ['was', 'not']
o hasnt ['has', 'not']
o theatre ['theater']
o cancelled ['canceled']
o organisation ['organization']
o labour ['labor']
o favourite ['favorite']
o travelling ['traveling']
o washingtons ['washington']
o marylands ['maryland']
o chinas ['china']
o russias ['russia']
o irans ['iran']
o dulles ['dulle']


In [19]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████| 843220/843220 [00:01<00:00, 610473.53it/s]

Found embeddings for 14.16% of vocab
Found embeddings for  85.10% of all text


In [20]:
oov[:40]

[('to', 11788929),
 ('a', 10516905),
 ('and', 10341307),
 ('of', 9694380),
 ('—', 1517033),
 ('000', 618627),
 ('10', 370043),
 ('30', 308695),
 ('11', 232210),
 ('12', 190189),
 ('20', 172134),
 ('15', 165409),
 ('–', 130187),
 ('703', 126301),
 ('900', 113356),
 ('25', 110584),
 ('13', 107690),
 ('18', 106101),
 ('2012', 105997),
 ('14', 104874),
 ('50', 101847),
 ('16', 101128),
 ('100', 96989),
 ('2014', 94357),
 ('17', 91094),
 ('500', 89634),
 ('2016', 82488),
 ('24', 81104),
 ('2013', 79778),
 ('…', 79766),
 ('40', 76906),
 ('21', 75482),
 ('19', 74826),
 ('2015', 72897),
 ('22', 71464),
 ('28', 65849),
 ('2011', 65395),
 ('23', 64728),
 ('27', 63606),
 ('26', 63521)]

We see that although we improved on the amount of embeddings found for all our text from 89% to 99%. Lets check the oov words again 

Looks good. No obvious oov words there we could quickly fix.
Thank you for reading and happy kaggling

# write out

In [25]:
with open('./clean_data/inverted_file.json','w') as f:
    f.write(json.dumps(invert))
with open('./clean_data/vocab','w') as f:
    for key in vocab:
        f.write('{0}\t{1}\n'.format(key,vocab[key]))

In [23]:
def check_coverage_large(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        la = ''.join([word[0].upper(),word[1:]])
        if(word in embeddings_index or la in embeddings_index):
            a[word] = word
            k += vocab[word]
        else:
            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [89]:
del invert['']

In [24]:
oov = check_coverage_large(vocab,embeddings_index)

100%|██████████| 843220/843220 [00:01<00:00, 755026.39it/s]

Found embeddings for 38.32% of vocab
Found embeddings for  96.68% of all text


In [95]:
oov[:20]

[('brexit', 84),
 ('deplorables', 77),
 ('sharknado', 45),
 ('f—', 45),
 ('tywanza', 28),
 ('selfie', 27),
 ('rapability', 25),
 ('rapable', 25),
 ('covfefe', 22),
 ('ooooooouuu', 21),
 ('schlonged', 20),
 ('pozdravleniya', 19),
 ('daesh', 18),
 ('ncis', 15),
 ('…', 15),
 ('adorabilis', 14),
 ('splatoon', 13),
 ('n—', 13),
 ('nastaliq', 12),
 ('whatd', 12)]